In [1]:
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.10:0.4.1 pyspark-shell' 

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import explode, col


sc = SparkContext('local', 'Spark SQL') 
spark = SQLContext(sc)

In [3]:
df = spark.read.format('com.databricks.spark.xml').options(rootTag='root').options(rowTag='course').load('reed_pv.xml')
 
df.printSchema()

root
 |-- crse: double (nullable = true)
 |-- days: string (nullable = true)
 |-- instructor: string (nullable = true)
 |-- place: struct (nullable = true)
 |    |-- building: string (nullable = true)
 |    |-- room: string (nullable = true)
 |-- reg_num: double (nullable = true)
 |-- sect: string (nullable = true)
 |-- subj: string (nullable = true)
 |-- time: struct (nullable = true)
 |    |-- end_time: string (nullable = true)
 |    |-- start_time: string (nullable = true)
 |-- title: string (nullable = true)
 |-- units: double (nullable = true)
 |-- xml_repository: string (nullable = true)



In [4]:
df.registerTempTable("XMLTABLE")

In [41]:
spark.sql("select crse, place.building from XMLTABLE").show(5, False)

+-----+--------+
|crse |building|
+-----+--------+
|211.0|
ELIOT  |
|344.0|
VOLLUM |
|431.0|
PHYSIC |
|431.0|
       |
|431.0|
BIOL   |
+-----+--------+
only showing top 5 rows



In [55]:
df = spark.sql("""
select
replace(replace(crse, chr(10)),chr(20)) crse,
replace(replace(days, chr(10)),chr(20)) days,
replace(replace(instructor, chr(10)),chr(20)) instructor,
replace(replace(place.building, chr(10)),chr(20)) place_building, 
rtrim(replace(replace(place.room, chr(10)),chr(20))) place_room,
rtrim(replace(replace(reg_num, chr(10)),chr(20))) reg_num,
replace(replace(sect, chr(10)),chr(20)) sect,
replace(replace(subj, chr(10)),chr(20)) subj,
replace(replace(time.start_time, chr(10)),chr(20)) time_start_time,
replace(replace(time.end_time, chr(10)),chr(20)) time_end_time,
replace(replace(title, chr(10)),chr(20)) title,
replace(replace(units, chr(10)),chr(20)) units,
replace(replace(xml_repository, chr(10)),chr(20)) xml_repository
from XMLTABLE 
""")

In [56]:
df.show(200, False)

+-----+-----+-----------+--------------+---------------------------------------------+-------+----+----+---------------+-------------+---------------------------------+-----+------------------------------------------------+
|crse |days |instructor |place_building|place_room                                   |reg_num|sect|subj|time_start_time|time_end_time|title                            |units|xml_repository                                  |
+-----+-----+-----------+--------------+---------------------------------------------+-------+----+----+---------------+-------------+---------------------------------+-----+------------------------------------------------+
|211.0|M-W  |Brightman  |ELIOT         |414                                          |10577.0|F01 |ANTH|03:10PM        |04:30        |Introduction to Anthropology     |1.0  |null                                            |
|344.0|T-Th |Makley     |VOLLUM        |120                                          |20573.0|S01 |ANTH|

AnalysisException: 'Only one generator allowed per select clause but found 4: explode(place.building AS `building`), explode(place.room AS `room`), explode(time.start_time AS `start_time`), explode(time.end_time AS `end_time`);'